In [529]:
import requests
from bs4 import BeautifulSoup

In [530]:
import lxml.html as lh
import pandas as pd
import csv

Бургундия (Дижон) 072800
Бордо 075100

In [531]:
start_date = '31.12.1969'
end_date = '31.12.1971'

In [550]:
url=f'http://pogoda-service.ru/archive_gsod_res.php?country=FR&station=075100&datepicker_beg={start_date}&datepicker_end={end_date}&bsubmit=%D0%9F%D0%BE%D1%81%D0%BC%D0%BE%D1%82%D1%80%D0%B5%D1%82%D1%8C'

In [551]:
page = requests.get(url)

In [552]:
doc = lh.fromstring(page.content)

In [553]:
tr_elements = doc.xpath('//tr')

In [554]:
[len(T) for T in tr_elements[:12]]

[8]

In [537]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"ÐÐ°ÑÐ°"
2:"ÐÐ°ÐºÑÐ¸Ð¼Ð°Ð»ÑÐ½Ð°Ñ ÑÐµÐ¼Ð¿ÐµÑÐ°ÑÑÑÐ°"
3:"ÐÐ¸Ð½Ð¸Ð¼Ð°Ð»ÑÐ½Ð°Ñ ÑÐµÐ¼Ð¿ÐµÑÐ°ÑÑÑÐ°"
4:"Ð¡ÑÐµÐ´Ð½ÑÑ ÑÐµÐ¼Ð¿ÐµÑÐ°ÑÑÑÐ°"
5:"ÐÑÐ¼Ð¾ÑÑÐµÑÐ½Ð¾Ðµ Ð´Ð°Ð²Ð»ÐµÐ½Ð¸Ðµ"
6:"Ð¡ÐºÐ¾ÑÐ¾ÑÑÑ Ð²ÐµÑÑÐ°"
7:"ÐÑÐ°Ð´ÐºÐ¸"
8:"Ð­ÑÑÐµÐºÑÐ¸Ð²Ð½Ð°Ñ ÑÐµÐ¼Ð¿ÐµÑÐ°ÑÑÑÐ°"


In [538]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=8:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [539]:
[len(C) for (title,C) in col]

[0, 0, 0, 0, 0, 0, 0, 0]

In [540]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [541]:
df.columns = ['date', 'max', 'min', 'average', 'atm', 'wind', 'rain', 'effect_t']

In [542]:
df.head()

,date,max,min,average,atm,wind,rain,effect_t


In [543]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      0 non-null      float64
 1   max       0 non-null      float64
 2   min       0 non-null      float64
 3   average   0 non-null      float64
 4   atm       0 non-null      float64
 5   wind      0 non-null      float64
 6   rain      0 non-null      float64
 7   effect_t  0 non-null      float64
dtypes: float64(8)
memory usage: 124.0 bytes


In [544]:
df.to_excel(f'pogoda_{start_date}.xlsx', sheet_name=f'{start_date}')